# Use bart model to get the subtract

training data from CORNEL NEWSROOM:https://lil.nlp.cornell.edu/newsroom/index.html, the format is jsonl.gz    
I use it to train the model   
data after process is avliable on:https://drive.google.com/file/d/1bn43rT6v-YOyGUEO_VydYWRAWkFU6MLf/view?usp=sharing

## preprocess the training data

In [ ]:
import json
import gzip as gz

path = "train.jsonl.gz"
data = []

with gz.open(path) as f:
    for ln in f:
        obj = json.loads(ln)
        data.append(obj)
        
#data[5:]
#data[0]['summary']

In [ ]:
import pandas as pd
text = []
summary = []
for i in data:
    text.append(i['text'])
    summary.append(i['summary'])
    
summary_df = pd.DataFrame()
summary_df['text'] = text
summary_df['summary'] = summary

summary_df[5:]

In [ ]:
# use the last 30000 records data
sum_df = summary_df[:30000]
sum_df.to_csv('sum.csv',index = 'False')
textl = []
#replace \n
for i in range(0,30000):
    ii = "".join([s for s in text[i].splitlines(True) if s.strip()])
    iii = ii.replace('\n','')
    textl.append(iii)
    
#print(textl[2])

#replace \r\n
sumar = []
for i in sumr:
    #ii = "".join([s for s in text[i].splitlines(True) if s.strip()])
    iii = i.replace('\r\n','')
    sumar.append(iii)
    
print(sumar[1])
sumr = summary[:30000]
summ_df = pd.DataFrame()
summ_df['text'] = textl
summ_df['summary'] = sumr
summ_df.to_csv('sum2.csv',index = 'False')

#data for training
for i in range(len(textl)):
    with open('s2.txt', 'a') as writers: # 打开文件
        writers.write(textl[i] + '\t'+ sumar[i] + '\n')

# use bart model get subtract

package  
! pip install datasets  
! pip install transformers  
! pip install rouge-score  

In [ ]:
#prepare the dataset
from datasets import Dataset

class Data:
    def __init__(self, data_path, tokenizer):
        self.path = data_path
        self.max_input_length = 1024
        self.max_target_length = 150
        # self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)
        self.tokenizer = tokenizer
        
    def preprocess(self, train_scale=0.8):
        with open(self.path,'r') as f:
            raw_data = f.readlines()
        print(f"=======data_len: {len(raw_data)}")
        start = int(len(raw_data)*train_scale)
        print(f"======train_len: {start}")
        
        raw_train_data = raw_data[:start]
        raw_test_data = raw_data[start:]
        raw_train_test_data = {'train':{'id':[],'document':[],'summary':[]}, \
                               'test':{'id':[],'document':[],'summary':[]}}        
        
        for i,item in enumerate(raw_train_data):
            if len(item.split('\t')) != 3:
                continue
            url,text,label = item.split('\t')
            raw_train_test_data['train']['id'].append(i)
            
            # document is the train data, summary is the substract label
            raw_train_test_data['train']['summary'].append(label.strip())
            raw_train_test_data['train']['document'].append(text.strip())

        for j,item in enumerate(raw_test_data):
            if len(item.split('\t')) != 3:
                continue
            url,text,label = item.split('\t')
            raw_train_test_data['test']['id'].append(i+j+1)
            raw_train_test_data['test']['summary'].append(label.strip())
            raw_train_test_data['test']['document'].append(text.strip())
        
        def preprocess_function(examples):
            
            inputs = examples['document']
            model_inputs = self.tokenizer(inputs, max_length = self.max_input_length, padding = 'max_length', truncation=True)
            
            with self.tokenizer.as_target_tokenizer():
                labels = self.tokenizer(examples['summary'], max_length = self.max_target_length, padding = 'max_length', truncation = True)
            model_inputs['labels'] = labels['input_ids']
            return model_inputs
        
        train_dataset = Dataset.from_dict(raw_train_test_data['train'])
        test_dataset = Dataset.from_dict(raw_train_test_data['test'])
        tokenized_train_dataset = train_dataset.map(preprocess_function)
        tokenized_test_dataset = test_dataset.map(preprocess_function)
        return tokenized_train_dataset, tokenized_test_dataset

In [ ]:
#load the model
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartForConditionalGeneration
from transformers import AutoTokenizer

#checkpoint = "distilbart-xsum-9-6"
checkpoint = "sshleifer/distilbart-xsum-9-6"
model = BartForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
#evaluating metric use the rouge_score
import numpy as np
from rouge_score import rouge_scorer, scoring
import nltk

nltk.download('punkt')
  


def compute(predictions, references, rouge_types=None, use_agregator=True, use_stemmer=False):
    if rouge_types is None:
        rouge_types = ["rouge1", "rougeL"]

    scorer = rouge_scorer.RougeScorer(rouge_types=rouge_types, use_stemmer=True)
    if use_agregator:
        aggregator = scoring.BootstrapAggregator()
    else:
        scores = []

    for ref, pred in zip(references, predictions):
        score = scorer.score(ref, pred)
        if use_agregator:
            aggregator.add_scores(score)
        else:
            scores.append(score)

    if use_agregator:
        result = aggregator.aggregate()
    else:
        result = {}
        for key in scores[0]:
            result[key] = list(score[key] for score in scores)

    return result


#metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    #prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    #result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()

In [ ]:
#Configure Hyperparameters
batch_size = 1
args = Seq2SeqTrainingArguments( \
    "/Users/fusi/Desktop/ds/individual_project/demo/model", \
    evaluation_strategy = 'steps', \
    learning_rate = 5e-5, \
    per_device_train_batch_size = batch_size, \
    per_device_eval_batch_size = batch_size, \
    weight_decay = 0.01, \
    save_steps = 200, \
    #save_total_limit = 5, \
    num_train_epochs = 250, \
    predict_with_generate = True, \
    eval_steps = 200, \
    logging_first_step=True, \
    report_to="none")

In [ ]:
#use transformers api to train the data
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model, padding=True)
data = Data('s2.txt', tokenizer)
tokenized_train_dataset, tokenized_test_dataset = data.preprocess()
trainer =Seq2SeqTrainer( \
                       model, \
                       args, \
                       train_dataset = tokenized_train_dataset, \
                       eval_dataset = tokenized_test_dataset, \
                       data_collator = data_collator, \
                       tokenizer = tokenizer, \
                       compute_metrics = compute_metrics)

In [ ]:
trainer.train()

In [ ]:
def predict(sentence):
    inputs = tokenizer([sentence],max_length = 1024, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], num_beams=70, max_length=500,min_length=50,early_stopping=True)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return ' '.join(summary)

In [ ]:
l = []
pre = [] 
with open('summary.txt','r') as f :
  for line in f.readlines():
    line = line.strip('\n')
    l.append(line)
print(len(l))
for i in l:
  p = predict(i)
  pre.append(p)
    
df.to_csv('sum.csv')